Use this notebook for all ad hoc autoqa serverless runs

# Imports & Variables

In [ ]:
import requests
import concurrent.futures
import asyncio
import nest_asyncio
import re
import numpy as np
import json
import ast
import pandas as pd
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
import os
import tiktoken

endpoint_id = 'k9fanpyhc2z8ya'
url = f"https://api.runpod.ai/v2/{endpoint_id}/run"
headers = {"Authorization": "Bearer <token>", "Content-Type": "application/json"}

# Data - Transcripts and Prompts

Using Priceline only as an example - adjust transcript and prompt import process based on client

In [57]:
# Transcript Import and Processing

# Folder where your JSON files are located
folder_path = r'C:\Users\belie\OneDrive\Documents\GitHub\AutoQA-FunctionDev\Priceline\Redacted Calls & Json'  # <-- change this to your actual folder

# Store all reconstructed transcripts
transcripts = []
filenames = []

# Iterate through each file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.json'):
        file_path = os.path.join(folder_path, filename)
        
        with open(file_path, 'r') as f:
            data = json.load(f)
        
        # Extract the transcript
        transcript = data.get('Transcript', [])
        
        # Build the output
        output_lines = []
        for entry in transcript:
            participant_id = entry.get('ParticipantId', '')
            content = entry.get('Content', '')
            if participant_id and content:
                output_lines.append(f"{participant_id}: {content}")
        
        # Join with newlines
        final_output = "\n".join(output_lines)
        
        # Append to the transcripts list
        transcripts.append(final_output)
        filenames.append(filename)


In [58]:
# Prompt Import
df_prompts = pd.read_excel(r"Priceline\B2B_AI_Sales_QA_Form Prompts - Copy.xlsx", sheet_name="Prompts")
df_prompts.head()

,Category #,Category,Section Score Weight,Question #,Question,Auto Fail,Question Type,Instructions to Evaluators,Question Score Weight,Answers,Scoring,Automation,Notes,Prompt Context,Evaluation Criteria,Scoring Guideline,Combined Prompt
0,1,Greeting & Verifying Travel Info,0.313,1.1,Was the agent prepared for the call by answeri...,NaN,Single Selection,Select NO if the agent took longer than 5 seco...,0.063,YES / NO,"YES: 10, NO: 0",Generative AI,NaN,Determine whether the agent was prepared to ha...,Agent must answer the call within 5 seconds of...,Yes: Agent answers promptly and engages the ca...,Prompt Context: Determine whether the agent wa...
1,1,mpollock@amplifai... - Default Di...,0.313,1.2,Did the agent use an appropriate greeting stat...,NaN,Single Selection,Depending on the Affiliate Source code and SKI...,0.063,"Yes, Guest-Reservation Greeting / Yes, NON Gue...","Yes (any): 10, No: 0","Contact Lens category ""B2B_Sales_GR_Greetings""...",Need Contact Lens category to be present in me...,Determine whether the agent used the correct g...,Greeting must match one of the approved script...,Yes: Guest-Reservation Greeting: Agent uses ap...,Prompt Context: Determine whether the agent us...
2,1,Greeting & Verifying Travel Info,0.313,1.3,Did agent use the additional TPV reply/script ...,Y,Single Selection,Agents must use the approved Third Party Verbi...,0.063,YES / NO,"YES: 10, NO: 0",Generative AI,NaN,Determine whether the agent used the required ...,Agent must use one of the approved phrasings b...,Yes: Agent uses exact TPV script after a trigg...,Prompt Context: Determine whether the agent us...
3,1,Greeting & Verifying Travel Info,0.313,1.4,Did the agent capture all the Verify Travel de...,NaN,Single Selection,Verified check-in/checkout dates and days of ...,0.063,Poor / Good / Excellent,"Poor: 0, Good: 5, Excellent: 10",Generative AI,NaN,Determine whether the agent captured all requi...,Agent must gather the following information fr...,Poor: Key details are missing or inaccurate\nG...,Prompt Context: Determine whether the agent ca...
4,1,Greeting & Verifying Travel Info,0.313,1.5,Before proceeding to the Urgency Statement & P...,NaN,Single Selection,"After greeting, agents must verify & summarize...",0.063,Positive / Neutral / Negative \n\nPositive: Ac...,"Positive: 10, Neutral: 5, Negative: 0",Generative AI,NaN,Determine whether the agent confirmed the trav...,Agent must clearly summarize all verified trav...,Positive: Agent clearly and accurately summari...,Prompt Context: Determine whether the agent co...


## System & User Prompts

In [59]:
system_prompt = f"""
Answer questions based on the interaction between a call-center agent and a customer.
Ensure the output follows the JSON format below:
{{
    "Question #": "[Question Number]",
    "Answer": "[Yes/No/NA]",
    "Justification": "[Provide your justification based on the interaction]"
}}
Interaction:
{max(transcripts, key=len)}
"""

user_prompt = ""
for i, question in enumerate(df_prompts['Combined Prompt'].to_list(), 1):
    user_prompt += f"""
    Question {i}: {question}
    Answer the following in the format provided:
    {{
        "Question #": "{i}",
        "Answer": "[Yes/No/NA]",
        "Justification": "[Provide your justification based on the interaction]"
    }}
    """

## Decide Context Window

In [129]:
def count_tokens(text):
    encoder = tiktoken.encoding_for_model("gpt-4")
    tokens = encoder.encode(text)
    return len(tokens)

tokens = count_tokens(f"{system_prompt}\n{user_prompt}")
ALLOWED_TOKEN_LENGTHS = [1024, 2048, 4096, 8192, 16384, 32768, 65536]
if tokens > ALLOWED_TOKEN_LENGTHS[-1]:
    MAX_TOKENS = ALLOWED_TOKEN_LENGTHS[-1]
for allowed_len in ALLOWED_TOKEN_LENGTHS:
    if tokens <= allowed_len:
        MAX_TOKENS = allowed_len
        break

print(f"Token count of system and user prompt based on largest transcript: {tokens}")
print(f"Max tokens dynamically set to: {MAX_TOKENS}")

Token count of system and user prompt based on largest transcript: 10152
Max tokens dynamically set to: 16384


# Processing

## Job Submission

In [138]:
# Make note of initial number of jobs completed on endpoint
response_health = requests.get(f'https://api.runpod.ai/v2/{endpoint_id}/health', headers=headers)
initial_jobs_completed = response_health.json()["jobs"]['completed']
print(f"Initial jobs completed: {initial_jobs_completed}")
# For each transcript submit a job and store the returned job id
job_ids = []
for t in transcripts:
    system_prompt = f"""
    Answer questions based on the interaction between a call-center agent and a customer.
    Ensure the output follows the JSON format below:
    {{
        "Question #": "[Question Number]",
        "Answer": "[Yes/No/NA]",
        "Justification": "[Provide your justification based on the interaction]"
    }}
    Interaction:
    {t}
    """

    # Payload
    payload = {"input": {
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        "sampling_params": {"temperature": 0.7, "max_tokens": MAX_TOKENS}
    }}

    # Submit the job
    response = requests.post(url, headers=headers, json=payload)
    if response.status_code != 200:
        print("Failed to submit job:", response.text)
        job_ids.append(None)
    else:
        response_json = response.json()
        job_id = response_json["id"]
        job_ids.append(job_id)
        print(f"Job submitted. ID: {job_id}")

Initial jobs completed: 2135
Job submitted. ID: cabf703b-966e-43dd-97a3-5a5a5a919ff9-u1
Job submitted. ID: 26d5f7fd-25da-45ef-8afc-055477c32310-u2
Job submitted. ID: 198a29e8-82d4-4ae2-95c7-8f2a9590205d-u1
Job submitted. ID: edcecbfe-0e2a-47a9-be49-7bc992fa4de0-u2
Job submitted. ID: 3b7b14f8-aa32-41f5-8f8c-fe05d1e7e001-u2
Job submitted. ID: 8556bbdf-262d-4e17-9725-a28c93d71fb3-u1
Job submitted. ID: 75720b71-a7d7-45ad-825a-86745169d2c4-u2
Job submitted. ID: 11aebc7b-fa81-4986-80f8-6463bd65a0d5-u2
Job submitted. ID: ddf649c2-a8e1-4200-a94e-467945b2038c-u1
Job submitted. ID: 09bf237c-e4d8-43cc-95b2-a2273aee2a93-u1
Job submitted. ID: 3ffa6ea0-1fe3-4058-a0d8-9f1fd116dfc1-u2
Job submitted. ID: db87e01e-e52d-46d1-bbc7-4c1b3d8bf73c-u1
Job submitted. ID: a8deee21-9221-4f5d-bedd-17566582fe37-u1
Job submitted. ID: 0bbfc97a-9d6d-41aa-a866-b2673cef3265-u2
Job submitted. ID: a73f3868-52d4-436f-918d-85309df5c6f1-u1
Job submitted. ID: 15a9430d-f2c8-408b-a514-a7d24b6ecf3e-u1
Job submitted. ID: 877665b5

## Completion Check

In [139]:
# Loop until all jobs are complete or until 30 minutes have passed
break_point = 0
complete = False

while (complete == False) and (break_point < 1800):
    response_health = requests.get(f'https://api.runpod.ai/v2/{endpoint_id}/health', headers=headers)
    current_jobs_completed = response_health.json()["jobs"]['completed']
    print(f"Current jobs completed: {current_jobs_completed}")
    # Check if all jobs are complete
    if current_jobs_completed - initial_jobs_completed >= len(job_ids):
        print("All jobs are complete.")
        complete = True
        break
    time.sleep(30)  # Sleep for 30 seconds
    break_point += 30

if break_point >= 1800:
    print("Timed out waiting for jobs to complete.")

Current jobs completed: 2139
Current jobs completed: 2140
Current jobs completed: 2163
Current jobs completed: 2185
All jobs are complete.


### Completion Check Alternate - use if there are other jobs running on the endpoint

In [ ]:
def check_job_status(endpoint_id, job_id):
    """
    Check the status of a RunPod job.
    
    Args:
        endpoint_id: Your RunPod endpoint ID
        job_id: The ID of the job to check
        
    Returns:
        Dictionary containing job status and results (if complete)
    """
    url = f"https://api.runpod.ai/v2/{endpoint_id}/status/{job_id}"
    headers = {"Authorization": f"Bearer <token>"}
    
    response = requests.get(url, headers=headers)
    return response.json()

# Iterate over the job IDs and retrieve the number of completed jobs
def check_all_jobs_completed(jobs):
    completed = 0
    for j in jobs:
        status = check_job_status(endpoint_id, j)
        if status.get('status') == 'COMPLETED':
            print(f"Job {j} is complete.")
            completed += 1
    if completed == len(jobs):
        print("All jobs are complete.")
        return True
    else:
        print(f"Jobs completed: {completed}/{len(jobs)}")
        print("\n")
        return False
    
complete = False
break_point = 0
while break_point < 1800:
    complete = check_all_jobs_completed(job_ids)
    if complete:
        break
    time.sleep(30)  # Sleep for 30 seconds
    break_point += 30

Job a4815344-bbfc-4610-b4de-71c4c1c9fb20-u1 is complete.
Job 7bc8b4e1-d634-4b4e-85f5-edaa0e66369a-u2 is complete.
Job 22e51593-3aec-43b0-bf2c-a953ba25b52c-u2 is complete.
All jobs are complete.


## Response Acquisition and Validation

In [ ]:
# Job status check
def check_job_status(endpoint_id, job_id):
    """
    Check the status of a RunPod job.
    
    Args:
        endpoint_id: Your RunPod endpoint ID
        job_id: The ID of the job to check
        
    Returns:
        Dictionary containing job status and results (if complete)
    """
    url = f"https://api.runpod.ai/v2/{endpoint_id}/status/{job_id}"
    headers = {"Authorization": f"Bearer <token>"}
    
    response = requests.get(url, headers=headers)
    return response.json()

# Validation
def extract_jsons_from_response(raw_response):  # Extracts JSON content from the response text.
    # Function to extract content inside <think>...</think> tags
    def extract_think_content(response_text):
        """Extracts content inside <think>...</think> and other remaining text."""
        think_match = re.search(r'<think>(.*?)</think>', response_text, re.DOTALL)
        think_content = think_match.group(1).strip() if think_match else "No structured thought content available."
        remaining_text = re.sub(r'<think>.*?</think>', '', response_text, flags=re.DOTALL).strip()
        
        return think_content, remaining_text

    # Clean content and convert to dictionary
    def clean_and_dict(text):
        # Remove all non-JSON content before the JSON object starts and clean up
        cleaned_text = re.sub(r'^[^\{]*\{', '{', text, count=1)
        return ast.literal_eval(cleaned_text)

    # Extract the think content and remaining response
    think_content, final_ans = extract_think_content(raw_response)

    # Find all JSON-like blocks in the remaining text
    json_pattern = r'\{.*?\}'  # Regex to match all JSON blocks (start with '{' and end with '}')
    json_matches = re.findall(json_pattern, final_ans, flags=re.DOTALL)

    # Clean and convert each JSON string to a dictionary
    json_dicts = []
    for json_str in json_matches:
        cleaned_str = re.sub(r'^[^\{]*\{', '{', json_str, count=1)  # Clean each JSON block if needed
        json_dict = clean_and_dict(cleaned_str)
        json_dicts.append(json_dict)

    return json_dicts

In [141]:
# Iterate through all job ids, fetch the results, validate results, and store in appropriate data structure (dict for valid responses, list for failure)
valid_jobs = {}
retry_jobs = []
for j in range(len(job_ids)):
    status = check_job_status(endpoint_id, job_ids[j])
    if status.get('status') == 'COMPLETED':
        try:
            tokens = status.get('output')[0].get('choices')[0].get('tokens')[0]
            jsons = extract_jsons_from_response(tokens)
            if len(jsons) == len(df_prompts['Combined Prompt'].to_list()):
                print(f"Job {job_ids[j]} - Transcript {filenames[j]} completed successfully with {len(jsons)} JSON objects.")
                valid_jobs[filenames[j]] = jsons
            else:
                print(f"Job {job_ids[j]} - Transcript {filenames[j]} returned an unexpected number of JSON objects.")
                retry_jobs.append(j) # Only storing the index and not the job id, because job ids and transcripts are linked by index
        except Exception as e:
            print(f"Error processing job {job_ids[j]}: {e}")
            retry_jobs.append(j)
    else:
        print(f"Job {job_ids[j]} - Transcript {filenames[j]} is not completed yet. Status: {status.get('status')}")
        retry_jobs.append(j)
        

Job cabf703b-966e-43dd-97a3-5a5a5a919ff9-u1 - Transcript 04f1498f-05ac-48a6-ba4d-e805a49bb744_analysis_redacted_2025-04-25T14_08_46Z.json completed successfully with 17 JSON objects.
Job 26d5f7fd-25da-45ef-8afc-055477c32310-u2 - Transcript 0b95cf63-9f48-474a-813b-fc700b7775ed_analysis_redacted_2025-04-25T11_34_19Z.json returned an unexpected number of JSON objects.
Job 198a29e8-82d4-4ae2-95c7-8f2a9590205d-u1 - Transcript 103e4721-9aa8-43e9-a386-404bc51754e5_analysis_redacted_2025-04-24T12_23_27Z.json completed successfully with 17 JSON objects.
Job edcecbfe-0e2a-47a9-be49-7bc992fa4de0-u2 - Transcript 18135605-fa5e-4e2b-ba5d-a1d963c1f888_analysis_redacted_2025-04-24T14_10_21Z.json completed successfully with 17 JSON objects.
Job 3b7b14f8-aa32-41f5-8f8c-fe05d1e7e001-u2 - Transcript 21aff88e-ff10-4726-ac03-e3ef3b16b8d6_analysis_redacted_2025-04-23T18_43_22Z.json completed successfully with 17 JSON objects.
Job 8556bbdf-262d-4e17-9725-a28c93d71fb3-u1 - Transcript 235b798f-aa81-40fc-994f-a7

In [142]:
len(valid_jobs), len(retry_jobs)

(45, 5)

## Retry Invalid Jobs

Run this section as many times as needed until the retry_jobs list is emptied

In [143]:
# Make note of initial number of jobs completed on endpoint
response_health = requests.get(f'https://api.runpod.ai/v2/{endpoint_id}/health', headers=headers)
initial_jobs_completed = response_health.json()["jobs"]['completed']
print(f"Initial jobs completed: {initial_jobs_completed}")

job_ids = []
# For each inavlid job id, resubmit the job and store the returned job id
for t in retry_jobs:
    system_prompt = f"""
    Answer questions based on the interaction between a call-center agent and a customer.
    Ensure the output follows the JSON format below:
    {{
        "Question #": "[Question Number]",
        "Answer": "[Yes/No/NA]",
        "Justification": "[Provide your justification based on the interaction]"
    }}
    Interaction:
    {transcripts[t]}
    """

    # Payload
    payload = {"input": {
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        "sampling_params": {"temperature": 0.7, "max_tokens": MAX_TOKENS}
    }}

    # Submit the job
    response = requests.post(url, headers=headers, json=payload)
    if response.status_code != 200:
        print("Failed to submit job:", response.text)
        job_ids.append(None)
    else:
        response_json = response.json()
        job_id = response_json["id"]
        job_ids.append(job_id)
        print(f"Job submitted. ID: {job_id}")

Initial jobs completed: 2185
Job submitted. ID: 8e896796-f172-4060-b459-ef343184c801-u2
Job submitted. ID: f6c19726-5349-4511-bcce-6bca89c42f65-u1
Job submitted. ID: 4e48ed96-e806-465f-93f2-27d56d3cadf6-u1
Job submitted. ID: e4ae27e1-dc6d-40d1-b63f-faf235fc4038-u2
Job submitted. ID: 2bd8e13b-a178-4d8b-9cf6-49a3acc842e2-u2


In [ ]:
def check_job_status(endpoint_id, job_id):
    """
    Check the status of a RunPod job.
    
    Args:
        endpoint_id: Your RunPod endpoint ID
        job_id: The ID of the job to check
        
    Returns:
        Dictionary containing job status and results (if complete)
    """
    url = f"https://api.runpod.ai/v2/{endpoint_id}/status/{job_id}"
    headers = {"Authorization": f"Bearer <token>"}
    
    response = requests.get(url, headers=headers)
    return response.json()

# Iterate over the job IDs and retrieve the number of completed jobs
def check_all_jobs_completed(jobs):
    completed = 0
    for j in jobs:
        status = check_job_status(endpoint_id, j)
        if status.get('status') == 'COMPLETED':
            print(f"Job {j} is complete.")
            completed += 1
    if completed == len(jobs):
        print("All jobs are complete.")
        return True
    else:
        print(f"Jobs completed: {completed}/{len(jobs)}")
        print("\n")
        return False
    
complete = False
break_point = 0
while break_point < 1800:
    complete = check_all_jobs_completed(job_ids)
    if complete:
        break
    time.sleep(30)  # Sleep for 30 seconds
    break_point += 30

Jobs completed: 0/5


Job 8e896796-f172-4060-b459-ef343184c801-u2 is complete.
Job f6c19726-5349-4511-bcce-6bca89c42f65-u1 is complete.
Job 4e48ed96-e806-465f-93f2-27d56d3cadf6-u1 is complete.
Job e4ae27e1-dc6d-40d1-b63f-faf235fc4038-u2 is complete.
Job 2bd8e13b-a178-4d8b-9cf6-49a3acc842e2-u2 is complete.
All jobs are complete.


In [151]:
retry_jobs_cp = retry_jobs.copy()
retry_jobs = []
for j in range(len(job_ids)):
    status = check_job_status(endpoint_id, job_ids[j])
    if status.get('status') == 'COMPLETED':
        try:
            tokens = status.get('output')[0].get('choices')[0].get('tokens')[0]
            jsons = extract_jsons_from_response(tokens)
            if len(jsons) == len(df_prompts['Combined Prompt'].to_list()):
                print(f"Job {job_ids[j]} - Transcript {filenames[retry_jobs_cp[j]]} completed successfully with {len(jsons)} JSON objects.")
                valid_jobs[filenames[retry_jobs_cp[j]]] = jsons
                #valid_jobs[retry_jobs_cp[j]] = jsons
            else:
                print(f"Job {job_ids[j]} returned an unexpected number of JSON objects.")
                retry_jobs.append(j) # Only storing the index and not the job id, because job ids and transcripts are linked by index
        except Exception as e:
            print(f"Error processing job {job_ids[j]}: {e}")
            retry_jobs.append(j)
    else:
        print(f"Job {job_ids[j]} is not completed yet. Status: {status.get('status')}")
        retry_jobs.append(j)

Job 8e896796-f172-4060-b459-ef343184c801-u2 - Transcript 0b95cf63-9f48-474a-813b-fc700b7775ed_analysis_redacted_2025-04-25T11_34_19Z.json completed successfully with 17 JSON objects.
Job f6c19726-5349-4511-bcce-6bca89c42f65-u1 - Transcript 311a058e-f1a8-4cc1-a2b3-6c748873183b_analysis_redacted_2025-04-24T18_56_58Z.json completed successfully with 17 JSON objects.
Job 4e48ed96-e806-465f-93f2-27d56d3cadf6-u1 - Transcript 7c9a6d37-d6a1-4715-a17e-68739ba2e339_analysis_redacted_2025-04-25T12_23_49Z.json completed successfully with 17 JSON objects.
Job e4ae27e1-dc6d-40d1-b63f-faf235fc4038-u2 - Transcript 826b804e-7f6a-4ce5-8384-606dda0a4d6c_analysis_redacted_2025-04-24T17_39_22Z.json completed successfully with 17 JSON objects.
Job 2bd8e13b-a178-4d8b-9cf6-49a3acc842e2-u2 - Transcript e29539d9-da23-451b-8d74-d0719e7a4847_analysis_redacted_2025-04-24T15_47_15Z.json completed successfully with 17 JSON objects.


In [152]:
len(valid_jobs), len(retry_jobs)

(50, 0)

# Post Processing (Re-mapping)

In [ ]:
# Reset index to ensure row order is aligned
df_prompts = df_prompts.reset_index(drop=True)

# For each file/evaluation result in the dictionary
for file_name, evaluations in valid_jobs.items():
    # Extract answers and justifications as ordered lists
    answers = [entry.get('Answer', '') for entry in evaluations]
    justifications = [entry.get('Justification', '') for entry in evaluations]
    
    # column names
    answer_col_name = f'{file_name} - Answer'
    justification_col_name = f'{file_name} - Justification'
    
    # Initialize columns with empty strings in case the dict has fewer items than df rows
    df_prompts[answer_col_name] = ''
    df_prompts[justification_col_name] = ''

    # Fill in row-by-row
    for i, (answer, justification) in enumerate(zip(answers, justifications)):
        if i < len(df_prompts):
            df_prompts.at[i, answer_col_name] = answer
            df_prompts.at[i, justification_col_name] = justification


C:\Users\belie\AppData\Local\Temp\ipykernel_47736\3489209819.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_prompts[justification_col_name] = ''
C:\Users\belie\AppData\Local\Temp\ipykernel_47736\3489209819.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_prompts[answer_col_name] = ''
C:\Users\belie\AppData\Local\Temp\ipykernel_47736\3489209819.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining a

In [111]:
df_prompts

,Category #,Category,Section Score Weight,Question #,Question,Auto Fail,Question Type,Instructions to Evaluators,Question Score Weight,Answers,...,e83d4539-2e71-452c-96fa-1021ec0a9e41_analysis_redacted_2025-04-24T13_55_08Z.json - Answer,e83d4539-2e71-452c-96fa-1021ec0a9e41_analysis_redacted_2025-04-24T13_55_08Z.json - Justification,e9763ab3-8dbb-4e60-b4d9-47fa900648c1_analysis_redacted_2025-04-25T14_00_02Z.json - Answer,e9763ab3-8dbb-4e60-b4d9-47fa900648c1_analysis_redacted_2025-04-25T14_00_02Z.json - Justification,8 - Answer,8 - Justification,40 - Answer,40 - Justification,41 - Answer,41 - Justification
0,1,Greeting & Verifying Travel Info,0.313,1.1,Was the agent prepared for the call by answeri...,NaN,Single Selection,Select NO if the agent took longer than 5 seco...,0.063,YES / NO,...,Yes,The agent promptly greeted the caller without ...,Yes,The agent promptly greeted the caller and enga...,Yes,The agent promptly greeted the customer and en...,Yes,The agent promptly answered the call and greet...,Yes,The agent promptly answered the call and engag...
1,1,mpollock@amplifai... - Default Di...,0.313,1.2,Did the agent use an appropriate greeting stat...,NaN,Single Selection,Depending on the Affiliate Source code and SKI...,0.063,"Yes, Guest-Reservation Greeting / Yes, NON Gue...",...,Yes,The agent used the correct Guest-Reservation g...,Yes,The agent used the correct Non-Guest Reservati...,No,The agent did not use the full Guest-Reservati...,Yes,The agent used the correct Non-Guest Reservati...,Yes,The agent used the correct Guest-Reservation g...
2,1,Greeting & Verifying Travel Info,0.313,1.3,Did agent use the additional TPV reply/script ...,Y,Single Selection,Agents must use the approved Third Party Verbi...,0.063,YES / NO,...,No,The agent did not use the required Third Party...,No,The agent did not use the TPV script as the ho...,No,The agent did not use the Third-Party Verbiage...,No,The agent did not use the required Third Party...,NA,No trigger for Third Party Verbiage (TPV) was ...
3,1,Greeting & Verifying Travel Info,0.313,1.4,Did the agent capture all the Verify Travel de...,NaN,Single Selection,Verified check-in/checkout dates and days of ...,0.063,Poor / Good / Excellent,...,Yes,The agent successfully captured all required t...,No,The agent missed capturing the number of child...,Yes,The agent confirmed all required details inclu...,No,The agent did not capture all required travel ...,Yes,The agent collected and confirmed all required...
4,1,Greeting & Verifying Travel Info,0.313,1.5,Before proceeding to the Urgency Statement & P...,NaN,Single Selection,"After greeting, agents must verify & summarize...",0.063,Positive / Neutral / Negative \n\nPositive: Ac...,...,Yes,The agent confirmed the travel details before ...,No,The agent did not summarize the travel details...,No,The agent restated the details but did not con...,No,The agent did not confirm all travel details b...,Yes,The agent summarized the travel details before...
5,2,PPS POWER Presentation,0.188,2.1,Did the agent create effective urgency through...,NaN,Single Selection,Agents must create urgency at least once after...,0.063,Nailed it perfectly! / Not quite there / Misse...,...,No,The agent did not create urgency or use urgenc...,Yes,The agent mentioned limited availability due t...,Yes,The agent used phrases like 'it's already been...,No,The agent did not create urgency or highlight ...,Yes,The agent used urgency appropriately by mentio...
6,2,PPS POWER Presentation,0.188,2.2,Did the agent ask effective probing questions ...,NaN,Single Selection,1. Effective Probing & fact finding The agent ...,0.063,"Yes, agent confirmed room type preference and...",...,No,The agent did not ask probing questions or use...,No,The agent did not ask probing questions about ...,No,The agent did not ask probing questions about ...,No,The agent did not ask probing questions about ...,Yes,The agent asked about the number of rooms and ...
7,2,PPS POWER Presentation,0.188,2.3,Did 